## Spatial interpolation and approximation II: splines
Resources:

* [
GRASS GIS overview and manual](http://grass.osgeo.org/grass72/manuals/index.html)
*  [Recommendations](data_acquisition.html#commands)
and [tutorial](http://www4.ncsu.edu/~akratoc/GRASS_intro/)
how to use wxGUI from the first assignment



Text files with color rules:

* [deviations_color.txt](data/deviations_color.txt)
* [precip_color.txt](data/precip_color.txt)



### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In startup pannel set GIS Data Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For Project location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for Accessible mapset create a new mapset (called e.g. HW_interpolation_2).
Click Start GRASS.


Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

In [ ]:
# a proper directory is already set, download files
import urllib
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/deviations_color.txt", "deviations_color.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/precip_color.txt", "precip_color.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/deviations_color.txt", "deviations_color.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/precip_color.txt", "precip_color.txt")

Download all text files with color rules (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.


### Interpolate elevation raster from points using splines with different tension
Compute aspect simultaneously with interpolation and 
evaluate impact of tension by using tension=40 (default), tension=10 and tension=160.

In [ ]:
gs.parse_command('g.region', region="rural_1m", res="1", flags='pg')
gs.run_command('v.surf.rst', input="elev_lid792_randpts", elevation="elev_rstdef_1m", zcolumn="value", aspect="asp_rstdef_1m", segmax="30", npmin="140")
gs.run_command('v.surf.rst', input="elev_lid792_randpts", elevation="elev_rstt10_1m", aspect="asp_rstt10_1m", zcolumn="value", tension="10", segmax="30", npmin="140")
gs.run_command('v.surf.rst', input="elev_lid792_randpts", elevation="elev_rstt160_1m", aspect="asp_rstt160_1m", zcolumn="value", tension="160", segmax="30", npmin="140")

Compare the interpolated elevation surfaces using aspect maps.
Change the aspect color table to grey aspect.
Save images for your report.

In [ ]:
gs.run_command('r.colors', map="asp_rstdef_1m", color="aspect")
gs.run_command('r.colors', map="asp_rstt10_1m", color="aspect")
gs.run_command('r.colors', map="asp_rstt160_1m", color="aspect")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_rstdef_1m")
gs.run_command('d.rast', map="asp_rstdef_1m")
Image(filename="map.png")
gs.run_command('d.rast', map="asp_rstt10_1m")
Image(filename="map.png")
gs.run_command('d.rast', map="asp_rstt160_1m")
Image(filename="map.png")
gs.run_command('d.vect', map="elev_lid792_randpts", size="1", color="red")
Image(filename="map.png")

Or use 3D views of elev_rstdef_1m, elev_rstt10_1m, elev_rstt160_1m, 
make sure you switch off the aspect rasters and save the 3 images for your report.

### Compute elevation raster and deviations vector point map
For different values of smoothing
compare deviation stats for smoothing 0.1 and 10.
Find root mean square deviation rmse.

In [ ]:
gs.run_command('v.surf.rst', input="elev_lid792_randpts", elevation="elev_rstdef_1mb", zcolumn="value", smooth="0.1", deviations="elev_rstdef_devi", segmax="30", npmin="140")
gs.run_command('v.build', map="elev_rstdef_devi")
gs.run_command('v.surf.rst', input="elev_lid792_randpts", elevation="elev_rstsm10_1mb", zcolumn="value", smooth="10", deviations="elev_rstsm10_devi", segmax="30", npmin="140")
gs.run_command('v.build', map="elev_rstsm10_devi")
gs.read_command('v.info', map="elev_rstdef_devi", flags='c')
gs.parse_command('v.univar', map="elev_rstdef_devi", column="flt1", type="point", flags='g')
gs.parse_command('r.info', map="elev_rstdef_1mb", flags='g')
gs.read_command('v.info', map="elev_rstsm10_devi", flags='c')
gs.parse_command('v.univar', map="elev_rstsm10_devi", column="flt1", type="point", flags='g')
gs.parse_command('r.info', map="elev_rstsm10_1mb", flags='g')

Compute and display deviations maps using same color table.
You need to use custom color table to see the results well.
Note that we are interpolating here the deviations, not the given elevations.

In [ ]:
gs.run_command('v.surf.rst', input="elev_rstdef_devi", elevation="elev_rstdef_devi", zcolumn="flt1", segmax="30", npmin="140")
gs.run_command('v.surf.rst', input="elev_rstsm10_devi", elevation="elev_rstsm10_devi", zcolumn="flt1", segmax="30", npmin="140")

Apply the downloaded color table [deviations_color.txt](data/deviations_color.txt)
to the deviation raster.
Optionally, to view the results in 3D use "elev_rstdef_1mb" for elevation (switch off everything else)
and drape the deviations maps as color.

In [ ]:
gs.run_command('r.colors', map="elev_rstsm10_devi", rules="deviations_color.txt")
gs.run_command('r.colors', map="elev_rstdef_devi", raster="elev_rstsm10_devi")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_rstdef_devi")
gs.run_command('d.rast', map="elev_rstsm10_devi")
gs.run_command('d.legend', raster="elev_rstsm10_devi", at="2,50,2,6")
Image(filename="map.png")

### Compute predictive error of interpolation
Compute predictive error of interpolation for each point
using cross-validation (no raster output, only points with pred. errors).

In [ ]:
gs.run_command('v.surf.rst', input="elev_lid792_randpts", zcolumn="value", cvdev="elev_rstdef_cv", npmin="120", segmax="35", flags='c')
gs.run_command('v.build', map="elev_rstdef_cv")
gs.parse_command('v.univar', map="elev_rstdef_cv", column="flt1", type="point", flags='g')

Compute raster map of predictive errors and identify locations
where the sampling is inadequate.
Optionally, to view the result in 3D use "elev_rstdef_1mb" for elevation (switch off everything else)
and drape the crossvalidation map "elev_rstdef_cv" as color.

In [ ]:
gs.run_command('v.surf.rst', input="elev_rstdef_cv", elevation="elev_rstdef_cv", zcolumn="flt1")
gs.run_command('r.colors', map="elev_rstdef_cv", raster="elev_rstsm10_devi")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_rstdef_cv")
gs.run_command('d.vect', map="elev_rstdef_cv", size="2")
gs.run_command('d.legend', raster="elev_rstdef_cv", at="2,50,2,6")
Image(filename="map.png")

### Interpolate precipitation with influence of topography

Set the 3D region (read the man page for _g.region_).
We set _tbres_ to high value - we have just a single level
because we are not computing the 3D raster (see lecture for more details).

In [ ]:
gs.parse_command('g.region', raster="elev_state_500m", flags='pg')
gs.parse_command('g.region', t="2000", b="0", tbres="2000", res3="500", flags='p3g')

Compute precipitation raster map without influence of elevation
(with segmax=700 segmentation is not performed so
interpolation function is computed using all points at once).
We will use mask during the interpolation.

In [ ]:
gs.run_command('r.mask', raster="ncmask_500m")
gs.read_command('v.info', map="precip_30ynormals", flags='c')
gs.run_command('v.surf.rst', input="precip_30ynormals", elevation="precip_annual_500m", zcolumn="annual", segmax="700")

Use the downloaded the color table [precip_color.txt](data/precip_color.txt).
Zoom to computational region when displaying the result.

In [ ]:
gs.run_command('r.colors', map="precip_annual_500m", rules="precip_color.txt")
gs.run_command('d.erase')
gs.run_command('d.rast', map="precip_annual_500m")
gs.run_command('d.legend', raster="precip_annual_500m", at="2,30,2,5", range="970,2400")
Image(filename="map.png")

Compute precipitation raster map with elevation.


There is both 3D voxel output and 2D raster output - we want the 2D raster output (cross_output).
Optionally to view the results in 3D,
switch off everything except for elev_state_500m and precip_30ynormals_3d,
switch to 3D, set(type in) viewer height at 300000, z-exag at 6, fine res=1,
use precip_anntopo_500m for color, set icon size for points - sphere, 5000.
Display the result and save the image for the report.

In [ ]:
gs.read_command('v.info', map="precip_30ynormals_3d", flags='c')
gs.run_command('v.vol.rst', input="precip_30ynormals_3d", cross_input="elev_state_500m", cross_output="precip_anntopo_500m", maskmap="elev_state_500m", wcolumn="annual", zscale="90", segmax="700")
gs.run_command('r.colors', map="precip_anntopo_500m", raster="precip_annual_500m")
gs.run_command('d.rast', map="precip_anntopo_500m")
Image(filename="map.png")

Try to explain how was elevation used for the precipitation raster interpolation.

After you are finished, remove mask.

In [ ]:
gs.run_command('r.mask', flags='r')

In [ ]:
# end the GRASS session
os.remove(rcfile)